In [2]:
preprocess_erp('3', '1')

----------------- load data ------------------
../data/bids/sub-3/eeg/sub-3_task-pitch_run-1_eeg.vhdr


/home/letitiayhho/ipykernel_3363158/1305382155.py:56: RuntimeWarning: The unit for channel(s) Aux1 has changed from NA to V.
  raw = read_raw_bids(bids_path, verbose = False)
/home/letitiayhho/ipykernel_3363158/1305382155.py:56: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['leog', 'reog', 'Aux1']
  raw = read_raw_bids(bids_path, verbose = False)
/home/letitiayhho/ipykernel_3363158/1305382155.py:56: RuntimeWarning: Not setting position of 1 stim channel found in montage:
['Aux1']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = read_raw_bids(bids_path, verbose = False)


Used Annotations descriptions: ['11', '12', '13', '21', '22', '23', '31', '32', '33']
----------------- re-reference eye electrodes to become bipolar EOG ------------------
Reading 0 ... 28283249  =      0.000 ...  5656.650 secs...
----------------- run PREP pipeline ------------------


KeyboardInterrupt: 

In [1]:
#!/usr/bin/env python3

#SBATCH --account=pi-hcn1
#SBATCH --time=00:40:00 # 40 min enough for most
#SBATCH --partition=bigmem
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=64G # 96 enough for most
#SBATCH --mail-type=all
#SBATCH --mail-user=letitiayhho@uchicago.edu
#SBATCH --output=logs/preprocess_erp-%j.log

import numpy as np
import os.path as op
import os
import sys

cwd = os.getcwd()
sys.path.append(cwd)
print(sys.path)

from pprint import pformat
import argparse
# EEG utilities
import mne
from mne.preprocessing import ICA, create_eog_epochs
from pyprep.prep_pipeline import PrepPipeline
# BIDS utilities
from mne_bids import BIDSPath, read_raw_bids
from util.io.bids import DataSink

# constants
BIDS_ROOT = '../data/bids'
DERIV_ROOT = op.join(BIDS_ROOT, 'derivatives')
ERP_PASSBAND = (0.1, 40)
TASK = 'pitch'
TMIN = -0.3
TMAX = 0.3

def preprocess_erp(sub, run):
    '''
    Parameters
    ----------
    sub : str
        Subject ID as in BIDS dataset
    '''
    # load data
    print('----------------- load data ------------------')
    bids_path = BIDSPath(
        root = BIDS_ROOT,
        subject = sub,
        task = TASK,
        run = run,
        datatype = 'eeg'
        )
    print(bids_path)
    raw = read_raw_bids(bids_path, verbose = False)
    events, event_ids = mne.events_from_annotations(raw)

    # re-reference eye electrodes to become bipolar EOG
    print('----------------- re-reference eye electrodes to become bipolar EOG ------------------')
    raw.load_data()
    def reref(dat):
        dat[0,:] = (dat[1,:] - dat[0,:])
        return dat
    raw = raw.apply_function(
        reref,
        picks = ['leog', 'Fp2'],
        channel_wise = False
    )
    raw = raw.apply_function(
        reref,
        picks = ['reog', 'Fp1'],
        channel_wise = False
    )
    
    raw = raw.set_channel_types({'leog': 'eog', 'reog': 'eog'})

    # run PREP pipeline (notch, exclude bad chans, and re-reference)
    print('----------------- run PREP pipeline ------------------')
    raw, events = raw.resample(int(4*ERP_PASSBAND[1]), events = events) # resample to 1200 Hz
    np.random.seed(int(sub))
    lf = raw.info['line_freq']
    prep_params = {
        "ref_chs": "eeg",
        "reref_chs": "eeg",
        "line_freqs": np.arange(lf, ERP_PASSBAND[1], lf)
    }
    prep = PrepPipeline(
        raw,
        prep_params,
        raw.get_montage(),
        ransac = False,
        random_state = int(sub)
        )
    prep.fit()
    
    # Extract data from PREP
    print('----------------- Extract data from PREP ------------------')
    prep_eeg = prep.raw_eeg # get EEG channels from PREP
    prep_non_eeg = prep.raw_non_eeg # get non-EEG channels from PREP
    raw_data = np.concatenate((prep_eeg.get_data(), prep_non_eeg.get_data())) # combine data from the two
    
    # Create info object for post-PREP data
    print('Create info object for post-PREP data')
    new_ch_names = prep_eeg.info['ch_names'] + prep_non_eeg.info['ch_names']
    raw = raw.reorder_channels(new_ch_names) # modify the channel names on the original raw data
    raw_info = raw.info # use the modified info from the original raw data object
     
    # Combine post-prep data and new info
    print('Create new raw object')
    raw = mne.io.RawArray(raw_data, raw_info) # replace original raw object

    ## now prepare non-epoched data for ERP analysis
    # identify bad ICs on weakly highpassed data
    print('----------------- Epoch data for ERP analysis ------------------')
    raw_for_ica = raw.copy().filter(l_freq = 0.1, h_freq = None)
    epochs_for_ica = mne.Epochs(
        raw_for_ica,
        events, # same events as FFR epochs
        tmin = TMIN,
        tmax = TMAX, # only prestim
        event_id = event_ids,
        baseline = None,
        preload = True
    )
    epochs.drop_bad(reject = dict(eeg = 35e-6))

    print('----------------- identify bad ICs on weakly highpassed data ------------------')
    ica = ICA(n_components = 15, random_state = 0)
    ica.fit(epochs_for_ica, picks = ['eeg', 'eog'])
    eog_indices, eog_scores = ica.find_bads_eog(epochs_for_ica, threshold = 1.96)
    ica.exclude = eog_indices
    
    # filter to desired bandwidth and remove bad ICs
    print('----------------- filter to desired bandwidth and remove bad ICs ------------------')
    raw = raw.filter(*FFR_PASSBAND)
    epochs_for_erp = mne.Epochs(
        raw,
        epochs.events, # same events as FFR epochs
        tmin = TMIN,
        tmax = TMAX, # only prestim
        event_id = event_ids,
        baseline = None,
        preload = True
    )
    
    # apply ICA
    print('----------------- apply ICA ------------------')
    ica.apply(epochs_for_erp) # transforms in place
    # now we no longer need EOG channels
    epochs_for_erp = epochs_for_erp.drop_channels('leog')
    epochs_for_erp = epochs_for_erp.drop_channels('reog')
    # and save
    sink = DataSink(DERIV_ROOT, 'erp')
    erp_fpath = sink.get_path(
        subject = sub,
        task = TASK,
        run = run,
        desc = 'forERP',
        suffix = 'epo',
        extension = 'fif.gz'
    )
    
    # Save data for ERP analysis
    print(f'Saving epochs for ERP analysis to: {erp_fpath}')
    epochs_for_erp.save(erp_fpath, overwrite = True)

    # generate a report
    print('----------------- generate a report ------------------')
    report = mne.Report(verbose = True)
    report.parse_folder(op.dirname(ffr_fpath), pattern = '*epo.fif.gz', render_bem = False)
    if ica.exclude:
        fig_ica_removed = ica.plot_components(ica.exclude, show = False)
        report.add_figure(
            fig_ica_removed,
            title = 'Removed ICA Components',
            section = 'ICA'
        )
    bads = prep.noisy_channels_original
    html_lines = []
    for line in pformat(bads).splitlines():
        html_lines.append('<br/>%s' % line)
    html = '\n'.join(html_lines)
    report.add_html(html, title = 'Interpolated Channels', section = 'channels')
    report.add_html(epochs.info._repr_html_(), title = 'Epochs Info (FFR)', section = 'info')
    report.add_html(epochs_for_erp.info._repr_html_(), title = 'Epochs Info (ERP)', section = 'info')
    report.save(op.join(sink.deriv_root, 'sub-%s.html'%sub), overwrite = True)


['/project/hcn1/.conda/envs/mne/lib/python311.zip', '/project/hcn1/.conda/envs/mne/lib/python3.11', '/project/hcn1/.conda/envs/mne/lib/python3.11/lib-dynload', '', '/project/hcn1/.conda/envs/mne/lib/python3.11/site-packages', '/project2/hcn1/pitch_tracking_attention/analysis']
